In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large, NASNetMobile

seed = 199 
tf.random.set_seed(seed)

In [ ]:
!bash get_data.sh

kagglecatsanddogs_3 100%[===================>] 786.68M   110MB/s    in 8.7s    
Downloading...
From: https://drive.google.com/u/0/uc?id=10NIeg2v6b9SzBBkqzbxUT_xTTgzIjsmv
To: /content/Dataset/101_ObjectCategories.tar.gz
132MB [00:01, 84.0MB/s]
Unziping... OK


In [ ]:
base_path = '/content/Dataset'
SHAPE = (224,224,3)
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(validation_split=0.1)


train_generator = datagen.flow_from_directory(
            base_path,
            target_size = (SHAPE[0], SHAPE[1]),
            batch_size = batch_size,
            class_mode =  'binary',
            shuffle = True,
            seed = seed,
            classes = ['Cats', 'NonCats'],
            subset = 'training'
    )

test_generator = datagen.flow_from_directory(
            base_path,
            target_size = (SHAPE[0], SHAPE[1]),
            batch_size = batch_size,
            class_mode =  'binary',
            shuffle = True,
            seed = seed,
            classes = ['Cats', 'NonCats'],
            subset = 'validation'
    )

Found 19480 images belonging to 2 classes.
Found 2163 images belonging to 2 classes.


In [ ]:
def get_model(base_model, input_shape):

    inputs = tf.keras.Input(shape=input_shape)

    x = base_model(inputs, training=False)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=tf.keras.metrics.BinaryAccuracy()) 

    return model

In [ ]:
MNV3Small = MobileNetV3Small(weights = 'imagenet', include_top = False, input_shape = SHAPE, minimalistic=True)
MNV3Small.trainable = False

model = get_model(MNV3Small, SHAPE)
model.summary()

4497408/4491912 [==============================] - 0s 0us/step
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
MobilenetV3small (Functional (None, 7, 7, 1024)        1031848   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 1,032,873
Trainable params: 1,025
Non-trainable params: 1,031,848
_________________________________________________________________


In [ ]:
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10
609/609 [==============================] - 469s 769ms/step - loss: 0.0512 - binary_accuracy: 0.9838 - val_loss: 0.1484 - val_binary_accuracy: 0.9191
Epoch 2/10
609/609 [==============================] - 471s 773ms/step - loss: 0.0479 - binary_accuracy: 0.9842 - val_loss: 0.1464 - val_binary_accuracy: 0.9214
Epoch 3/10
609/609 [==============================] - 476s 781ms/step - loss: 0.0457 - binary_accuracy: 0.9854 - val_loss: 0.1505 - val_binary_accuracy: 0.9172
Epoch 4/10
609/609 [==============================] - 477s 784ms/step - loss: 0.0463 - binary_accuracy: 0.9851 - val_loss: 0.1572 - val_binary_accuracy: 0.9145
Epoch 5/10
609/609 [==============================] - 475s 779ms/step - loss: 0.0446 - binary_accuracy: 0.9851 - val_loss: 0.1295 - val_binary_accuracy: 0.9325
Epoch 6/10
609/609 [==============================] - 478s 785ms/step - loss: 0.0445 - binary_accuracy: 0.9860 - val_loss: 0.1272 - val_binary_accuracy: 0.9413
Epoch 7/10
609/609 [====================